# Initialization

In [1]:
import logging, os, sys, time

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import backprojection as tdbp
from posarmctools import getNotebookLogger

sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia

# this is for the sceneReferencePoint.py module
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [2]:
%matplotlib qt5
eps = np.finfo(float).eps

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
logger = getNotebookLogger(logging.INFO)

In [5]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [6]:
lib = tdbp.Lib(libraryFilename)

# Load focusing configuration and data

In [7]:
# 2020 06 25
focusingDir = "/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/FOCUSING"
#ini = "2019_07_12_14_12_21.ini"
#ini = "2019_07_12_14_14_43.ini"
#ini = "2019_07_12_14_18_28.ini"
ini = "2019_07_12_14_12_21_center.ini"

focusing = tdbp.Focusing(os.path.join(focusingDir, ini), tag="Sa", alongTrack=0, pointsFile="py")

======= Conf
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_12_21
======= Track
======= Load analytic signal...
load /home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/OUT/2019_07_12_14_12_21/Sa_files_0_58_rampUp_hanning.npy
sa.shape (59000, 5000)
existing analytic signal found and loaded
======= Load antenna positions...
load /home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/OUT/2019_07_12_14_12_21/n_time_xyz_nav_a.npy
xa.shape (59000,)
xa_mean = 1326910.42, ya_mean = 7218872.38, za_mean = 449.61
======= Load Digital Terrain Elevation Data...
get dataset
build meshgrids
build rectBivariateSpline
======= Scene
buildSceneGroundRange
======= Set focusingParameters
Nf 5000, Naz 59000, phi_a_deg 10.0, azRes 0.086
range from 0.00m to 2499.95m, resolution = 0.5m, oversampled = 0.05m, 


## Compute scene elevations

In [ ]:
focusing.rebuildScene()

In [8]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
ax.plot(x, y, '.k')
ax.grid()
focusing.addPtsEpsg(ax, ['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])
ax.plot(focusing.scene.targetX, focusing.scene.targetY, 'o', label="scene.target")
ax.plot(focusing.track.refX, focusing.track.refY, 'D', label="track.ref")
#ax.plot(focusing.scene.refX, focusing.scene.refY, 'D', label="scene.ref")
#ax.plot(focusing.scene.closestApproach[0], focusing.scene.closestApproach[1], 'o')

# Focalization

In [9]:
t = time.time()

focusedImg  = np.zeros( (focusing.scene.nY, focusing.scene.nX), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )

lib.so.backProjectionOmpGroundRange_LETG(focusing.scene.X.reshape(focusing.scene.X.size), 
                                         focusing.scene.Y.reshape(focusing.scene.Y.size),
                                         focusing.sceneZ.reshape(focusing.sceneZ.size),
                                         focusing.r_over,
                                         focusing.sa.reshape(focusing.sa.size),
                                         focusing.xyz.reshape(focusing.xyz.size), 
                                         focusedImg.reshape(focusedImg.size),
                                         focusing.parameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg.shape = (800, 800)
execution time = 826.0886006355286


In [1]:
myImg = tdbp.Img(focusedImg)
#min_dB = -34.70, max_dB = 67.19, med_dB = 28.79

NameError: name 'tdbp' is not defined

# Plot image

## imshow

In [11]:
cmap = 'jet'

In [14]:
left = focusing.conf.xMax
right = focusing.conf.xMin
bottom = focusing.conf.yMax
top = focusing.conf.yMin
extent = [left, right, bottom, top]
vmin = myImg.max_dB - 40 # 60 pour ground range, 40 pour slant range
vmax = myImg.max_dB
#vmin = myImg.med_dB - 15
#vmax = myImg.med_dB + 15

axTitle = f"{focusing.conf.data_date}\n{focusing.conf.groundOrSlant}, {focusing.conf.phi_a_deg:.1f}°"\
            + f", dx {focusing.conf.d_x}, dx {focusing.conf.d_y}"\
            + f", {focusing.conf.window}"

if 0:
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
    #im0 = ax0.imshow(myImg.img_dB, cmap=cmap, extent=extent)
    #im1 = ax1.imshow(myImg.box_dB(3), cmap=cmap, extent=extent)
    #im0 = ax0.imshow(myImg.img_dB, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
    #im1 = ax1.imshow(myImg.box_dB(3), cmap=cmap, extent=extent)
    dia.addColorBar(im0, ax0)
    dia.addColorBar(im1, ax1)
else:
    fig, ax0 = plt.subplots(1, 1)
    #im0 = ax0.imshow(myImg.img_dB, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
    #im0 = ax0.imshow(myImg.img_dB, cmap=cmap, extent=extent)
    im0 = ax0.imshow(myImg.img_dB, cmap=cmap, vmin=vmin, vmax=vmax)
    dia.addColorBar(im0, ax0)
    ax0.set_title(axTitle)
    ax0.set_aspect('equal')

In [ ]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
idx = 273
toPlot = myImg.img_dB.T[idx, :]
maxVal = np.amax(toPlot)
xAxis = np.arange(focusing.conf.xMax, focusing.conf.xMin, -focusing.conf.d_x)
yAxis = np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y)
plt.plot(yAxis, toPlot, label=f"coupe à x = {xAxis[idx]:.2f}")
plt.axhline(y=maxVal, color ="green", linestyle ="--") 
plt.axhline(y=maxVal - 3, color ="green", linestyle =":") 
plt.grid()
plt.legend()
plt.xlabel("y [m]")
plt.ylabel("[dB]")

## resample

In [ ]:
coef = 4
newImg = signal.resample(focusedImg, focusedImg.shape[0]*coef, axis=0)
newImg = signal.resample(newImg, focusedImg.shape[1]*coef, axis=1)
newMyImg = tdbp.Img(newImg)

In [ ]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
idxAlt = idx * coef
toPlot = newMyImg.img_dB.T[idxAlt, :]
maxVal = np.amax(toPlot)
xAxis = np.arange(focusing.conf.xMax, focusing.conf.xMin, -focusing.conf.d_x / coef)
yAxis = np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y / coef)
plt.plot(yAxis, toPlot, label=f"coupe à x = {xAxis[idxAlt]:.2f}")
plt.axhline(y = maxVal, color ="green", linestyle ="--") 
plt.axhline(y = maxVal - 3, color ="green", linestyle =":") 
plt.grid()
plt.legend()
plt.xlabel("y [m]")
plt.ylabel("[dB]")

In [ ]:
fig, ax0 = plt.subplots(1, 1, figsize=(10, 5))
im0 = ax0.imshow(newMyImg.img_dB, cmap=cmap, vmin=vmin, vmax=vmax)
dia.addColorBar(im0, ax0)

## pcolormesh box_filter

In [15]:
z = myImg.box_dB(3)

In [16]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet')
ax.set_aspect('equal')
#ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#ax.plot(focusing.conf.ptsEpsg['corner'][0], focusing.conf.ptsEpsg['corner'][1], 'og', markeredgecolor = 'black')

In [17]:
vmin = 0
vmax = 40

fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet', vmin=vmin, vmax=vmax)
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#focusing.addPtsEpsg(ax, ['corner'])
#focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

## Dynamic range

In [18]:
imgDyn = myImg.dynamicRange(dynRange=60, db=1, size=3)

min -28.8, max 16.2


In [24]:
fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(x, y, z, cmap=cmap, vmin=-10, vmax=40)
dia.addColorBar(im, ax)
ax.grid()
#focusing.addPtsEpsg(ax, ['c2'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

# Build GeoTIFF

In [21]:
tiffName = os.path.join(focusing.out_dir, focusing.conf.name + ".tif")

In [27]:
myTiff = tdbp.Tiff(tiffName, focusing, z, cmap='gray')

# Save focused image

In [23]:
myImg.save(focusing)

In [26]:
focusing.track.theta*180/np.pi

176.22486700487016